In [1]:
import tensorflow
import os
import numpy as np
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.applications.resnet import ResNet50, preprocess_input,ResNet101
from tensorflow.keras.applications import (
    InceptionV3, ResNet50, ResNet101, MobileNetV2,
    DenseNet121, VGG16, VGG19, DenseNet169,
    DenseNet201, Xception, NASNetLarge, NASNetMobile,
    EfficientNetB0, EfficientNetB1, EfficientNetB2,
    EfficientNetB3, EfficientNetB4, EfficientNetB5,
    EfficientNetB6, EfficientNetB7
)
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix,classification_report
from sklearn.metrics import roc_curve

2023-11-10 02:19:44.397025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-10 02:19:44.436854: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# Set dataset paths
ct_train_path = "/u/45/muhammu2/data/Desktop/group1/trainset/"
ct_validation_path = "/u/45/muhammu2/data/Desktop/group1/validation/"
ct_test_path = "/u/45/muhammu2/data/Desktop/group1/testset/"

# Number of classes depends on the dataset (2 for the binary classification)
ct_num_classes = 2

# Image size depends on the pretrained model (224 for InceptionV3)
np.random.seed(42)
image_size = 224
batch_size = 16
epoch_num = 15

# Define a data generator for training set with data augmentation
ct_train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

ct_train_generator = ct_train_datagen.flow_from_directory(
    ct_train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Define a data generator for validation set (without data augmentation)
ct_validation_datagen = image.ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

ct_validation_generator = ct_validation_datagen.flow_from_directory(
    ct_validation_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Define a data generator for testing set (without data augmentation)
ct_test_datagen = image.ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

ct_test_generator = ct_test_datagen.flow_from_directory(
    ct_test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

Found 240 images belonging to 2 classes.
Found 360 images belonging to 2 classes.
Found 480 images belonging to 2 classes.


In [3]:
# Assuming image_size is set to 224
input_shape = (224, 224, 3)  # Adjust according to the target size of your images

# Create different models for comparison and choose the best model
base_model = InceptionV3(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = ResNet50(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = ResNet101(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = MobileNetV2(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = DenseNet121(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = VGG16(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = VGG19(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = DenseNet169(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = DenseNet201(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = Xception(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
base_model = NASNetLarge(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = NASNetMobile(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB1(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB4(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB5(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB6(include_top=False, weights='imagenet', input_shape=input_shape)
base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=input_shape)

2023-11-10 02:19:46.187843: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-10 02:19:46.215902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-10 02:19:46.216145: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:

# Assuming image_size is set to 224
input_shape = (224, 224, 3)  # Adjust according to the target size of your images

# Create the ResNet50 base (without the top layer)
base_model = EfficientNetB7(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)

# Freeze the weights of the ResNet50 base
base_model.trainable = False

# Create the top layers for binary classification
top_model = Sequential()
top_model.add(Flatten())
top_model.add(Dense(64, activation='relu'))
top_model.add(Dense(ct_num_classes, activation='sigmoid'))  # Change to 'sigmoid' for binary classification

# Combine the base and top models
model = Sequential()
model.add(base_model)
model.add(top_model)

# Compile the combined model
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb7 (Functional  (None, 2560)              64097687  
 )                                                               
                                                                 
 sequential (Sequential)     (None, 2)                 164034    
                                                                 
Total params: 64261721 (245.14 MB)
Trainable params: 164034 (640.76 KB)
Non-trainable params: 64097687 (244.51 MB)
_________________________________________________________________


In [5]:
# Define model parameters
steps_per_epoch = ct_train_generator.samples // batch_size
validation_steps = ct_validation_generator.samples // batch_size

In [6]:
# Assuming you have already defined ct_train_generator, ct_validation_generator, steps_per_epoch, epoch_num, and validation_steps

# Train the model using the fit method
model.fit(
    ct_train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epoch_num,
    validation_data=ct_validation_generator,
    validation_steps=validation_steps,
    verbose=1
)


Epoch 1/15


2023-11-10 02:20:55.108257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2023-11-10 02:20:55.342907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-10 02:20:55.563292: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc49c3aa3d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-10 02:20:55.563313: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A2000 8GB Laptop GPU, Compute Capability 8.6
2023-11-10 02:20:55.567724: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-10 02:20:55.646093: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime 

15/15 [==============================] - 27s 962ms/step - loss: 2.9907 - accuracy: 0.7083 - val_loss: 0.8591 - val_accuracy: 0.2443
Epoch 2/15
15/15 [==============================] - 10s 658ms/step - loss: 0.7080 - accuracy: 0.5208 - val_loss: 0.6922 - val_accuracy: 0.7500
Epoch 3/15
15/15 [==============================] - 11s 716ms/step - loss: 0.6909 - accuracy: 0.7500 - val_loss: 0.6893 - val_accuracy: 0.7585
Epoch 4/15
15/15 [==============================] - 9s 627ms/step - loss: 0.6881 - accuracy: 0.7500 - val_loss: 0.6863 - val_accuracy: 0.7500
Epoch 5/15
15/15 [==============================] - 10s 711ms/step - loss: 0.6850 - accuracy: 0.7500 - val_loss: 0.6834 - val_accuracy: 0.7472
Epoch 6/15
15/15 [==============================] - 9s 644ms/step - loss: 0.6819 - accuracy: 0.7500 - val_loss: 0.6802 - val_accuracy: 0.7528
Epoch 7/15
15/15 [==============================] - 9s 574ms/step - loss: 0.6789 - accuracy: 0.7500 - val_loss: 0.6775 - val_accuracy: 0.7472
Epoch 8/15
15

In [9]:

# Assuming you have the true labels for the validation and test sets
true_labels_validation = ct_validation_generator.classes
true_labels_test = ct_test_generator.classes

# Assuming you have the predicted probabilities for the positive class (spoofed) from your model
predicted_probabilities_validation = model.predict(ct_validation_generator)[:, 1]
predicted_probabilities_test = model.predict(ct_test_generator)[:, 1]

# Compute the ROC curve on the validation set
fpr, tpr, thresholds = roc_curve(true_labels_validation, predicted_probabilities_validation, pos_label=1)

# Calculate EER on the validation set
eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Find the threshold where FAR equals FRR (EER threshold)
eer_threshold = thresholds[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Apply the EER threshold to the test set predictions
predictions_test = (predicted_probabilities_test > eer_threshold).astype(int)

# Calculate HTER on the test set
false_acceptance = np.sum((predictions_test == 1) & (true_labels_test == 0))
false_rejection = np.sum((predictions_test == 0) & (true_labels_test == 1))
total_samples_test = len(true_labels_test)

hter = (false_acceptance + false_rejection) / (2 * total_samples_test)

# Display EER and HTER
print(f"Equal Error Rate (EER) on validation set: {eer:.2%}")
print(f"Half Total Error Rate (HTER) for Face Anti-Spoofing: {hter:.2%}")

30/30 [==============================] - 6s 198ms/step
Equal Error Rate (EER) on validation set: 0.00%
Half Total Error Rate (HTER) for Face Anti-Spoofing: 8.33%
